# Tutorial with examples

In [1]:
from cmfa.fluxomics_data.reaction import Reaction, AtomPattern
from cmfa.fluxomics_data.reaction_network import ReactionNetwork
from cmfa.data_preparation import load_dataset_from_csv, import_fluxomics_dataset_from_json
from cmfa.fluxomics_data.emu import EMU, EMUReaction


In [2]:
import os

CUR_DIR = os.getcwd()
TEST_DIR = f'{CUR_DIR}/../data/test_data'

TEST_DIR

'/Users/te/Library/CloudStorage/OneDrive-Personal/Biosustain/cmfa/cmfa/../data/test_data'

In [3]:
path_flux = f'{TEST_DIR}/flux.csv'
path_ms_measurement = f'{TEST_DIR}/ms_measurements.csv'
path_reaction_network = f'{TEST_DIR}/reactions.csv'
path_tracer = f'{TEST_DIR}/tracers.csv'

model = load_dataset_from_csv(
    tracer_file=path_tracer,
    flux_measurement_file=path_flux,
    mid_measurement_file=path_ms_measurement,
    reaction_file=path_reaction_network)

<function reverse_stoichiometry at 0x1351ff420>


/Users/te/Library/CloudStorage/OneDrive-Personal/Biosustain/cmfa/cmfa/fluxomics_data/reaction_network.py:96: UserWarning: adding auto-generated compound id='B' name=None formula=None carbon_label=None fragment_id=None m_z=None
  warnings.warn(
/Users/te/Library/CloudStorage/OneDrive-Personal/Biosustain/cmfa/cmfa/fluxomics_data/reaction_network.py:96: UserWarning: adding auto-generated compound id='C' name=None formula=None carbon_label=None fragment_id=None m_z=None
  warnings.warn(
/Users/te/Library/CloudStorage/OneDrive-Personal/Biosustain/cmfa/cmfa/fluxomics_data/reaction_network.py:96: UserWarning: adding auto-generated compound id='D' name=None formula=None carbon_label=None fragment_id=None m_z=None
  warnings.warn(
/Users/te/Library/CloudStorage/OneDrive-Personal/Biosustain/cmfa/cmfa/fluxomics_data/reaction_network.py:96: UserWarning: adding auto-generated compound id='E' name=None formula=None carbon_label=None fragment_id=None m_z=None
  warnings.warn(
/Users/te/Library/CloudS

In [5]:
model.reaction_network.reactions

{Reaction id=R1, name=R1,stoichiometry={'A': {abc: -1.0}, 'B': {abc: 1.0}},reversible=False, ,
 Reaction id=R2, name=R2,stoichiometry={'B': {abc: -1.0}, 'D': {abc: 1.0}},reversible=True, ,
 Reaction id=R2_rev, name=R2_rev,stoichiometry={'B': {abc: 1.0}, 'D': {abc: -1.0}},reversible=False, ,
 Reaction id=R3, name=R3,stoichiometry={'B': {abc: -1.0}, 'C': {bc: 1.0}, 'E': {a: 1.0}},reversible=False, ,
 Reaction id=R4, name=R4,stoichiometry={'B': {abc: -1.0}, 'C': {de: -1.0}, 'D': {bcd: 1.0}, 'E': {a: 1.0, e: 1.0}},reversible=False, ,
 Reaction id=R5, name=R5,stoichiometry={'D': {abc: -1.0}, 'F': {abc: 1.0}},reversible=False, }

In [8]:
model.reaction_network.R2_rev

Reaction id=R2_rev, name=R2_rev,stoichiometry={'B': {abc: 1.0}, 'D': {abc: -1.0}},reversible=False, 

In [9]:
print(model.reaction_network.reaction_adjacency_matrix())

compound           A         B     C         D               E           F
pattern          abc       abc    bc  de   abc   bcd         a     e   abc
compound pattern                                                          
A        abc      []      [R1]    []  []    []    []        []    []    []
B        abc      []        []  [R3]  []  [R2]  [R4]  [R4, R3]    []    []
C        bc       []        []    []  []    []    []        []    []    []
         de       []        []    []  []    []  [R4]        []  [R4]    []
D        abc      []  [R2_rev]    []  []    []    []        []    []  [R5]
         bcd      []        []    []  []    []    []        []    []    []
E        a        []        []    []  []    []    []        []    []    []
         e        []        []    []  []    []    []        []    []    []
F        abc      []        []    []  []    []    []        []    []    []


In [21]:
def atom_mapping_to_reaction(emu: EMU, reaction: Reaction, reverse: bool=False) -> list:
    """
    Given an EMU and reaction information, returns an EMUReaction object.
    """
    try:
        atom_transitions = reaction.stoichiometry_input[emu.compound].keys()
    except:
        raise ValueError(f"Cannot find the EMU compound in the corresponding reaction")

    emu_reactions = []
    # Use emu to find matching atoms in given compounds.
    # TODO: What if the emu matches two same compounds in reaction
    for transition in atom_transitions:
        adjusted_indices = [i - 1 for i in emu.atom_number]
        try:
            matched_atoms = ''.join(transition[i] for i in adjusted_indices)
        except:
            raise ValueError(f"Cannot find atoms in compound {emu.compound} with indices {emu.atom_number_input}")

        # Extracting the stoichiometry for the specific EMU
        emu_stoichiometry = {}
        emu_reactants = {emu}
        for cpd, stoich in reaction.stoichiometry_input.items():
            # new_stoich = {}
            for pattern, coeff in stoich.items():
                if cpd == emu.compound: # Add the original EMU
                    emu_stoichiometry[emu.id] = -coeff if reverse else coeff

                elif (not reverse and coeff < 0) or (reverse and reaction.reversible and coeff > 0):
                    # For matched reactants or products in reverse reaction
                    reactant_matched_atom_number = _find_matchable_atoms(pattern, matched_atoms)
                    if len(reactant_matched_atom_number) != 0:
                        emu_id = f"{cpd}_{reactant_matched_atom_number}"
                        emu_reactants.add(EMU(id=emu_id, compound=cpd, atom_number_input=reactant_matched_atom_number))
                        # emu_stoichiometry.setdefault(cpd, {})[reactant_matched_atom_number] = -coeff if reverse else coeff
                        emu_stoichiometry[emu_id] = -coeff if reverse else coeff

        emu_reactions.append(EMUReaction(reaction_id=reaction.id, emu_stoichiometry=emu_stoichiometry))

    return emu_reactions, set(emu_reactants)

def _find_matchable_atoms(pattern: str, matched_atoms: str):
    
    """
    Finds matchable atoms between a pattern and matched_atoms, returning the positions as a string.
    """
    matched_pattern = [str(pattern.index(char) + 1) for char in matched_atoms if char in pattern]
    return ''.join(sorted(matched_pattern))


# Example Usage
product_emu = EMU(id="B_123", compound="B", atom_number_input="123")
reaction = Reaction(id='R4', name='R4', stoichiometry_input={'B': {"abc": -1.0}, 'C': {"de": -1.0}, 'D': {"bcd": 1.0}, 'E': {"a": 1.0, "e": 1.0}}, reversible=True)

emu_reaction = atom_mapping_to_reaction(product_emu, reaction, reverse=True)
print(emu_reaction)


([reaction id: R4, EMU stoichiometry: {'B_123': 1.0, 'D_12': -1.0, 'E_1': -1.0}], {EMU id: B_123, compound id: B, atom pattern: (1, 2, 3), EMU id: D_12, compound id: D, atom pattern: (1, 2), EMU id: E_1, compound id: E, atom pattern: (1,)})


In [11]:
def find_participated_reactions(cur_emu, MAM):
    """
    Identifies the reactions in which a given EMU participates within the Metabolite Adjacency Matrix (MAM).
    
    Parameters:
    - MAM: DataFrame representing the Metabolite Adjacency Matrix with multi-index columns.
    - cur_emu: The current EMU object, expected to have at least a 'compound' attribute.
    
    Returns:
    - A DataFrame filtered to show only the reactions (columns) in which the current EMU's compound participates.
      The cells are boolean, indicating whether the reaction is non-empty (True) or not (False).
    """
    # Extract reactions for the current EMU's compound
    participated_reactions = MAM.xs(cur_emu.compound, level=0, axis=1).map(
        lambda x: x if isinstance(x, list) and len(x) > 0 else [])
    
    all_participated_reactions = set(item for sublist in participated_reactions.values.flatten() for item in sublist)
    return all_participated_reactions


In [16]:
import pandas as pd
import re
from collections import deque
from cmfa.fluxomics_data.emu import EMUMap

def decompose_network(
    target_EMU: EMU, reaction_network: ReactionNetwork
) -> pd.DataFrame:
    """
    Return a EMU map with a list of EMUs based on the target compound from the reaction network. 
    """
    MAM = reaction_network.reaction_adjacency_matrix()
    queue = deque([target_EMU])
    visited = set()
    emu_maps = {}
    emu_cpd_list = {}

    while queue:
        # Keep poping new EMU added from previous round.
        cur_emu = queue.popleft()
        # print(f"Current EMU: {cur_emu.compound}, have visited {visited}")
        if cur_emu in visited:
            continue
        visited.add(cur_emu)

        # Add new EMU map if the EMU size is not added
        emu_size = cur_emu.size
        if emu_size not in emu_maps:
            emu_maps[emu_size] = []
            emu_cpd_list[emu_size] = []
        
        # Find the reaction that is participated in the current emu
        participated_reactions = find_participated_reactions(cur_emu, MAM)
        new_map = []
        emus = []
        for r in participated_reactions:
            # Handle forward and reverse case
            # if r.endswith('_rev'):
            #     new_emu_reactions, new_emus = atom_mapping_to_reaction(cur_emu, reaction_network.find_reaction_by_id(r[:-4]), reverse=True)
            # else:
            new_emu_reactions, new_emus = atom_mapping_to_reaction(cur_emu, reaction_network.find_reaction_by_id(r), reverse=False)
            # print(new_emu_reactions, new_emus)
            new_map.append(new_emu_reactions)
            emus.append(new_emus)
            # TODO: Find equivalent EMUs
            for e in new_emus:
                if e not in visited and e not in queue:
                    queue.append(e)
                if e not in emu_cpd_list[emu_size]:
                    emu_cpd_list[emu_size] += [e]
        
        if len(new_map) > 0:
            # emu_maps[emu_size].setdefault(cur_emu, []).append(new_map)
            for i in new_map:
                emu_maps[emu_size] += i
            
    return EMUMap(emu_map=emu_maps, emu_set=emu_cpd_list)



In [17]:
f_emu = EMU(id='F_123', compound='F', atom_number_input='123')
emu_network, emu_set = decompose_network(f_emu, model.reaction_network)
emu_set

[reaction id: R5, EMU stoichiometry: {'D_123': -1.0, 'F_123': 1.0}] {EMU id: D_123, compound id: D, atom pattern: (1, 2, 3), EMU id: F_123, compound id: F, atom pattern: (1, 2, 3)}
[reaction id: R2, EMU stoichiometry: {'B_123': -1.0, 'D_123': 1.0}] {EMU id: B_123, compound id: B, atom pattern: (1, 2, 3), EMU id: D_123, compound id: D, atom pattern: (1, 2, 3)}
[reaction id: R4, EMU stoichiometry: {'B_23': -1.0, 'C_1': -1.0, 'D_123': 1.0}] {EMU id: B_23, compound id: B, atom pattern: (2, 3), EMU id: D_123, compound id: D, atom pattern: (1, 2, 3), EMU id: C_1, compound id: C, atom pattern: (1,)}
[reaction id: R1, EMU stoichiometry: {'A_123': -1.0, 'B_123': 1.0}] {EMU id: B_123, compound id: B, atom pattern: (1, 2, 3), EMU id: A_123, compound id: A, atom pattern: (1, 2, 3)}
[reaction id: R2_rev, EMU stoichiometry: {'B_123': 1.0, 'D_123': -1.0}] {EMU id: B_123, compound id: B, atom pattern: (1, 2, 3), EMU id: D_123, compound id: D, atom pattern: (1, 2, 3)}
[reaction id: R1, EMU stoichiometr

{3: [EMU id: D_123, compound id: D, atom pattern: (1, 2, 3),
  EMU id: F_123, compound id: F, atom pattern: (1, 2, 3),
  EMU id: B_123, compound id: B, atom pattern: (1, 2, 3),
  EMU id: B_23, compound id: B, atom pattern: (2, 3),
  EMU id: C_1, compound id: C, atom pattern: (1,),
  EMU id: A_123, compound id: A, atom pattern: (1, 2, 3)],
 2: [EMU id: B_23, compound id: B, atom pattern: (2, 3),
  EMU id: A_23, compound id: A, atom pattern: (2, 3),
  EMU id: D_23, compound id: D, atom pattern: (2, 3),
  EMU id: B_3, compound id: B, atom pattern: (3,),
  EMU id: C_1, compound id: C, atom pattern: (1,)],
 1: [EMU id: B_2, compound id: B, atom pattern: (2,),
  EMU id: C_1, compound id: C, atom pattern: (1,),
  EMU id: A_2, compound id: A, atom pattern: (2,),
  EMU id: D_2, compound id: D, atom pattern: (2,),
  EMU id: B_3, compound id: B, atom pattern: (3,),
  EMU id: A_3, compound id: A, atom pattern: (3,),
  EMU id: D_3, compound id: D, atom pattern: (3,)]}

In [18]:
emu_network

{3: [reaction id: R5, EMU stoichiometry: {'D_123': -1.0, 'F_123': 1.0},
  reaction id: R2, EMU stoichiometry: {'B_123': -1.0, 'D_123': 1.0},
  reaction id: R4, EMU stoichiometry: {'B_23': -1.0, 'C_1': -1.0, 'D_123': 1.0},
  reaction id: R1, EMU stoichiometry: {'A_123': -1.0, 'B_123': 1.0},
  reaction id: R2_rev, EMU stoichiometry: {'B_123': 1.0, 'D_123': -1.0}],
 2: [reaction id: R1, EMU stoichiometry: {'A_23': -1.0, 'B_23': 1.0},
  reaction id: R2_rev, EMU stoichiometry: {'B_23': 1.0, 'D_23': -1.0},
  reaction id: R2, EMU stoichiometry: {'B_23': -1.0, 'D_23': 1.0},
  reaction id: R4, EMU stoichiometry: {'B_3': -1.0, 'C_1': -1.0, 'D_23': 1.0}],
 1: [reaction id: R3, EMU stoichiometry: {'B_2': -1.0, 'C_1': 1.0},
  reaction id: R1, EMU stoichiometry: {'A_2': -1.0, 'B_2': 1.0},
  reaction id: R2_rev, EMU stoichiometry: {'B_2': 1.0, 'D_2': -1.0},
  reaction id: R1, EMU stoichiometry: {'A_3': -1.0, 'B_3': 1.0},
  reaction id: R2_rev, EMU stoichiometry: {'B_3': 1.0, 'D_3': -1.0},
  reaction 

In [22]:
import pandas as pd
from sympy import Symbol, Mul

def create_emu_stoichiometry_matrix(emu_network, emu_size):

    # Create matrix indices
    reactants = set()
    products = set()

    for reaction in emu_network[emu_size]:
        # Directly extend the sets without checking the length
        reactants.update([" * ".join(reaction.reactants)])
        products.update([" * ".join(reaction.products)])

    # Combine reactants and products into a single tuple for output
    indices = tuple(reactants.union(products))
    
    # Initialize the DataFrame with indices for both rows and columns
    matrix = pd.DataFrame(index=indices, columns=indices, data=[])
    # Populate the matrix
    for reaction in emu_network[emu_size]: 
        row_key, col_key = None, None  # Initialize keys
        
        for emu_id, stoich in reaction.emu_stoichiometry.items():
            if stoich < 0:  # Reactant
                for idx in indices:
                    # Find the matching reactant row
                    if emu_id in idx.split(' * '):
                        row_key = idx
                        
            elif stoich > 0:  # Product
                for idx in indices:
                    # Find the matching product column
                    if emu_id in idx.split(' * '):
                        col_key = idx
                
            # Update the matrix cell with reaction ID and stoichiometry
            if row_key and col_key:
                cell_value = abs(stoich) * reaction.flux_symbol
                matrix.at[row_key, col_key] = cell_value

    return matrix

matrix = create_emu_stoichiometry_matrix(emu_network, 1)
print(matrix)


     A_3     D_3     D_2     C_1  A_2         B_3         B_2
A_3  NaN     NaN     NaN     NaN  NaN      1.0*R1         NaN
D_3  NaN     NaN     NaN     NaN  NaN  1.0*R2_rev         NaN
D_2  NaN     NaN     NaN     NaN  NaN         NaN  1.0*R2_rev
C_1  NaN  1.0*R4     NaN     NaN  NaN         NaN         NaN
A_2  NaN     NaN     NaN     NaN  NaN         NaN      1.0*R1
B_3  NaN  1.0*R2  1.0*R4     NaN  NaN         NaN         NaN
B_2  NaN     NaN  1.0*R2  1.0*R3  NaN         NaN         NaN


In [33]:
def process_and_split_matrix(matrix):
    """
    Return A and B matrices according to the EMU algorithm.

    Parameters
    ----------
    matrix : pd.DataFrame
        The matrix to be processed and split, typically representing stoichiometries or interactions within an EMU network.

    Returns
    -------
    tuple of pd.DataFrame
        A tuple containing two pandas DataFrames after splitting and processing the input matrix. The first matrix's diagonal is adjusted to include non-NaN values from corresponding rows.
    """
    # Remove columns with all NaN values
    cleaned_matrix = matrix.dropna(axis=1, how="all")

    # Identify the row indices to split the matrix
    split_indices = [
        idx for idx in matrix.index if idx not in cleaned_matrix.columns
    ]

    # Split the matrix into two based on the identified indices
    matrix_A = (cleaned_matrix.loc[~cleaned_matrix.index.isin(split_indices)]).T
    matrix_B = (cleaned_matrix.loc[cleaned_matrix.index.isin(split_indices)]).T
    

    # Adjust the diagonal of the first matrix
    for i in range(len(matrix_A)):
        non_nan_values = (
            matrix_A.iloc[i].dropna().tolist()
            + matrix_B.iloc[i].dropna().tolist()
        )
        updated_values = 0
        for item in non_nan_values:
            # Copy the list to avoid modifying the original matrices
            updated_values += -1 * item.copy()

        # Assign the updated list to the diagonal element
        matrix_A.iat[i, i] = updated_values
        
    matrix_B = matrix_B.map(lambda x: -x if not pd.isnull(x) else x)

    return matrix_A, matrix_B

matrix_1, matrix_2 = process_and_split_matrix(matrix)
matrix_1


,D_3,D_2,C_1,B_3,B_2
D_3,-1.0*R2 - 1.0*R4,NaN,1.0*R4,1.0*R2,NaN
D_2,NaN,-1.0*R2 - 1.0*R4,NaN,1.0*R4,1.0*R2
C_1,NaN,NaN,-1.0*R3,NaN,1.0*R3
B_3,1.0*R2_rev,NaN,NaN,-1.0*R1 - 1.0*R2_rev,NaN
B_2,NaN,1.0*R2_rev,NaN,NaN,-1.0*R1 - 1.0*R2_rev


In [34]:
matrix_2

,A_3,A_2
D_3,NaN,NaN
D_2,NaN,NaN
C_1,NaN,NaN
B_3,-1.0*R1,NaN
B_2,NaN,-1.0*R1


In [35]:
import numpy as np
import sympy as sp

# Define the reactions as sympy symbols
R1, R2, R2_rev, R3, R4, R5 = sp.symbols('R1 R2 R2_rev R3 R4 R5')

# Define the reaction rates
reaction_rates = {R1: 100, R2: 110, R2_rev: 50, R3: 20, R4: 20, R5: 80}

def process_matrix(matrix_df, reaction_rates):
    # Define the lambda function for substitution and conversion
    process_element = lambda x: float(x.subs(reaction_rates)) if isinstance(x, sp.Expr) else 0
    
    # Apply the lambda function to each element in the DataFrame
    processed_matrix = matrix_df.map(process_element)

    # Convert the processed DataFrame to a NumPy array
    return processed_matrix.to_numpy(dtype=float)

# Applying the function to the matrices
matrix_np_1 = process_matrix(matrix_1, reaction_rates)
matrix_np_2 = process_matrix(matrix_2, reaction_rates)

print("Matrix 1:")
print(matrix_np_1)

print("\nMatrix 2:")
print(matrix_np_2)


Matrix 1:
[[-130.    0.   20.  110.    0.]
 [   0. -130.    0.   20.  110.]
 [   0.    0.  -20.    0.   20.]
 [  50.    0.    0. -150.    0.]
 [   0.   50.    0.    0. -150.]]

Matrix 2:
[[   0.    0.]
 [   0.    0.]
 [   0.    0.]
 [-100.    0.]
 [   0. -100.]]


In [36]:
Y = np.matrix([[1,0], [0,1]])

In [37]:
X = np.linalg.inv(matrix_np_1) @  matrix_np_2 @ Y

In [38]:
X

matrix([[0.8       , 0.2       ],
        [0.2       , 0.8       ],
        [0.06666667, 0.93333333],
        [0.93333333, 0.06666667],
        [0.06666667, 0.93333333]])